In [186]:
from pyspark.sql.functions import *

#import spark
from pyspark.sql import SparkSession
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 ass2 BNPL group 28")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

In [187]:
## read main and external dataset
curated_csv = spark.read.options(header = True).csv('../data/curated/full_data.csv')
postcode_SA2 = spark.read.options(header = True).csv('../data/tables/postcode_SA2.csv')
population = spark.read.options(header = True).csv('../data/tables/2021Census_G01_AUST_SA2.csv')
income = spark.read.options(header = True).csv('../data/tables/2021Census_G02_AUST_SA2.csv')

In [188]:
curated_csv.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- merchant_abn: string (nullable = true)
 |-- dollar_value: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_datetime: string (nullable = true)
 |-- consumer_id: string (nullable = true)
 |-- merchant_name: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- state: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- field: string (nullable = true)
 |-- revenue_level: string (nullable = true)
 |-- take_rate: string (nullable = true)



In [189]:
postcode_SA2.printSchema()

root
 |-- POA_CODE_2021: string (nullable = true)
 |-- SA2_CODE_2021: string (nullable = true)
 |-- SA2_NAME_2021: string (nullable = true)



In [190]:
postcode_SA2.count()

11858

In [191]:
population.printSchema()

root
 |-- SA2_CODE_2021: string (nullable = true)
 |-- Tot_P_M: string (nullable = true)
 |-- Tot_P_F: string (nullable = true)
 |-- Tot_P_P: string (nullable = true)
 |-- Age_0_4_yr_M: string (nullable = true)
 |-- Age_0_4_yr_F: string (nullable = true)
 |-- Age_0_4_yr_P: string (nullable = true)
 |-- Age_5_14_yr_M: string (nullable = true)
 |-- Age_5_14_yr_F: string (nullable = true)
 |-- Age_5_14_yr_P: string (nullable = true)
 |-- Age_15_19_yr_M: string (nullable = true)
 |-- Age_15_19_yr_F: string (nullable = true)
 |-- Age_15_19_yr_P: string (nullable = true)
 |-- Age_20_24_yr_M: string (nullable = true)
 |-- Age_20_24_yr_F: string (nullable = true)
 |-- Age_20_24_yr_P: string (nullable = true)
 |-- Age_25_34_yr_M: string (nullable = true)
 |-- Age_25_34_yr_F: string (nullable = true)
 |-- Age_25_34_yr_P: string (nullable = true)
 |-- Age_35_44_yr_M: string (nullable = true)
 |-- Age_35_44_yr_F: string (nullable = true)
 |-- Age_35_44_yr_P: string (nullable = true)
 |-- Age_45_54

In [192]:
income.printSchema()

root
 |-- SA2_CODE_2021: string (nullable = true)
 |-- Median_age_persons: string (nullable = true)
 |-- Median_mortgage_repay_monthly: string (nullable = true)
 |-- Median_tot_prsnl_inc_weekly: string (nullable = true)
 |-- Median_rent_weekly: string (nullable = true)
 |-- Median_tot_fam_inc_weekly: string (nullable = true)
 |-- Average_num_psns_per_bedroom: string (nullable = true)
 |-- Median_tot_hhd_inc_weekly: string (nullable = true)
 |-- Average_household_size: string (nullable = true)



In [193]:
## Drop unhelpful columns in both external dataset
population = population.select(col('SA2_CODE_2021'), col('Tot_P_P'))
income = income.select(col('SA2_CODE_2021'), col('Median_tot_prsnl_inc_weekly'))

## change data type
population = population.withColumn("Tot_P_P",population.Tot_P_P.cast('int'))
income = income.withColumn('Median_tot_prsnl_inc_weekly', income.Median_tot_prsnl_inc_weekly.cast('float'))

In [194]:
## merge SA2 population (sum of different sa2 areas) with postcode
pos_population = postcode_SA2.join(population, postcode_SA2.SA2_CODE_2021 == population.SA2_CODE_2021, "inner").drop(population.SA2_CODE_2021)
pos_population = pos_population.groupBy('POA_CODE_2021').sum('Tot_P_P')
pos_population = pos_population.withColumnRenamed('sum(Tot_P_P)', 'total_population')

pos_population.show(3)

+-------------+----------------+
|POA_CODE_2021|total_population|
+-------------+----------------+
|         2136|           67075|
|         4821|          212922|
|         2088|           28328|
+-------------+----------------+
only showing top 3 rows



In [195]:
## merge SA2 income (average personal income in different sa2 areas) with postcode
pos_income= postcode_SA2.join(income, postcode_SA2.SA2_CODE_2021 == income.SA2_CODE_2021, "inner").drop(income.SA2_CODE_2021)
pos_income = pos_income.groupBy('POA_CODE_2021').avg('Median_tot_prsnl_inc_weekly')
pos_income = pos_income.withColumn("avg_personal_income_weekly", round(pos_income["avg(Median_tot_prsnl_inc_weekly)"], 1))
pos_income = pos_income.drop('avg(Median_tot_prsnl_inc_weekly)')

pos_income.show(3)

+-------------+--------------------------+
|POA_CODE_2021|avg_personal_income_weekly|
+-------------+--------------------------+
|         2136|                     813.0|
|         4821|                     833.2|
|         2088|                    1487.0|
+-------------+--------------------------+
only showing top 3 rows



In [196]:
## join both income and population with postcode
pos_info = pos_population.join(pos_income, pos_population.POA_CODE_2021 == pos_income.POA_CODE_2021, 'inner').drop(pos_income.POA_CODE_2021)

pos_info = pos_info.select('POA_CODE_2021', 'total_population', 'avg_personal_income_weekly')

pos_info.show(3)

+-------------+----------------+--------------------------+
|POA_CODE_2021|total_population|avg_personal_income_weekly|
+-------------+----------------+--------------------------+
|         2136|           67075|                     813.0|
|         4821|          212922|                     833.2|
|         2088|           28328|                    1487.0|
+-------------+----------------+--------------------------+
only showing top 3 rows



In [197]:
## join pos_info with original dataset
curated_csv_new = curated_csv.join(pos_info, curated_csv.postcode == pos_info.POA_CODE_2021, "inner").drop(pos_info.POA_CODE_2021)

curated_csv_new.limit(3)

user_id,merchant_abn,dollar_value,order_id,order_datetime,consumer_id,merchant_name,user_name,address,state,postcode,gender,field,revenue_level,take_rate,total_population,avg_personal_income_weekly
13328,82448693380,186.46,05b84520-00a9-4e2...,2021-10-05,31719,Vulputate Dui Nec...,Catherine Simmons,0859 Scott Statio...,NSW,2034,Female,computer programm...,a,6.06,25132,1264.5
17506,75454398468,722.45,6e6e6b6c-9d1b-411...,2021-10-30,15418,Tempus Non Lacini...,Cynthia Young,7236 Bates Lock S...,NSW,2034,Female,tent and awning s...,b,4.9,25132,1264.5
13328,33604812025,135.82,06399e3b-e23b-4d7...,2021-08-02,31719,null,Catherine Simmons,0859 Scott Statio...,NSW,2034,Female,null,null,null,25132,1264.5


In [198]:
print('Number of transaction before join with pos_info:', str(curated_csv.count()))
print('Number of transaction after join with pos_info:', str(curated_csv_new.count()))


Number of transaction before join with pos_info: 8151372


Number of transaction after join with pos_info: 6719594
